## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-29-10-53-23 +0000,nypost,Texas teen who vanished on Christmas Eve feare...,https://nypost.com/2025/12/29/us-news/missing-...
1,2025-12-29-10-50-36 +0000,nyt,"For Zelensky, Just Keeping Trump Talking About...",https://www.nytimes.com/2025/12/29/world/europ...
2,2025-12-29-10-40-00 +0000,wsj,Yen Rises After BOJ Summary Signals Further Ra...,https://www.wsj.com/articles/dollar-trades-ste...
3,2025-12-29-10-38-29 +0000,bbc,Thailand accuses Cambodia of breaking newly si...,https://www.bbc.com/news/articles/c9w7wgrk01do...
4,2025-12-29-10-36-00 +0000,wsj,Swiss Asset Manager GAM Opposes Takeover of Ho...,https://www.wsj.com/business/deals/swiss-asset...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2443/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,22
107,new,14
8,zelensky,8
105,war,6
334,video,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
127,2025-12-28-19-01-49 +0000,nypost,Trump believes ‘we have makings of a deal’ to ...,https://nypost.com/2025/12/28/world-news/trump...,68
119,2025-12-28-20-02-25 +0000,nypost,Putin’s top negotiator posts video of himself ...,https://nypost.com/2025/12/28/world-news/putin...,61
135,2025-12-28-17-32-52 +0000,nyt,Zelensky to Meet With Trump at Mar-a-Lago Abou...,https://www.nytimes.com/2025/12/28/world/europ...,57
87,2025-12-29-00-34-51 +0000,nyt,Trump and Zelensky Meet to Iron Out Peace Plan...,https://www.nytimes.com/2025/12/28/us/politics...,54
1,2025-12-29-10-50-36 +0000,nyt,"For Zelensky, Just Keeping Trump Talking About...",https://www.nytimes.com/2025/12/29/world/europ...,53


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
127,68,2025-12-28-19-01-49 +0000,nypost,Trump believes ‘we have makings of a deal’ to ...,https://nypost.com/2025/12/28/world-news/trump...
13,42,2025-12-29-10-11-38 +0000,nyt,Parts of New York and New England Brace for Ic...,https://www.nytimes.com/2025/12/29/us/new-york...
47,32,2025-12-29-05-52-42 +0000,nypost,Minnesota Gov. Tim Walz on the defensive as fr...,https://nypost.com/2025/12/29/us-news/minnesot...
159,28,2025-12-28-14-17-31 +0000,cbc,"Brigitte Bardot, icon of French cinema turned ...",https://www.cbc.ca/news/entertainment/brigitte...
49,27,2025-12-29-05-28-22 +0000,nyt,China Will Hold Live-Fire Military Exercises A...,https://www.nytimes.com/2025/12/29/world/asia/...
29,25,2025-12-29-09-19-07 +0000,nypost,Bondi Beach hero Ahmed al Ahmed gives first in...,https://nypost.com/2025/12/29/world-news/bondi...
7,23,2025-12-29-10-30-00 +0000,wsj,A life-sciences job in Boston used to be a sur...,https://www.wsj.com/tech/biotech/ph-d-s-cant-f...
65,23,2025-12-29-03-03-18 +0000,nypost,"Interoceanic Train derails in southern Mexico,...",https://nypost.com/2025/12/28/world-news/inter...
79,22,2025-12-29-01-26-11 +0000,nypost,Heart-pounding video captures moment cop stops...,https://nypost.com/2025/12/28/us-news/heart-po...
50,21,2025-12-29-05-13-17 +0000,bbc,Five-year-old boy dies after getting caught in...,https://www.bbc.com/news/articles/c4g6gw30epvo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
